Basic Neural Net Code for https://www.kaggle.com/competitions/titanic

In [8]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import KFold

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
example = pd.read_csv("gender_submission.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [5]:
df = train.copy()

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Neural Net

In [7]:
def preprocess(df):
    df['_sex'] = np.select([df.Sex == 'male'],[0],1)
    df['_age'] = df.Age
    df['_fare'] = df.Fare
    df['_siblings'] = df.SibSp
    df['_par'] = df.Parch
    df['_class'] = df.Pclass
    return df
    
def calculate_columns(df, features):
    X = df[features]
    X = (X - X.mean()) / X.std()
    X = X.fillna(0)
    return X    

K-fold given set of hyperparameters

In [15]:
model = nn.Sequential(nn.Linear(6,10),
                      nn.ReLU(),
                      nn.Linear(10,10),
                      nn.ReLU(),
                      nn.Linear(10,10),   
                      nn.ReLU(),
                      nn.Linear(10,1),                         
                      nn.Sigmoid()
                     )
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

num_epochs = 2000

kf = KFold(n_splits=5,shuffle=True)
acc = []

features = ['_sex','_age','_fare','_siblings','_par','_class']
train = preprocess(train)

for i, (train_index, test_index) in enumerate(kf.split(train)):
    Xv = calculate_columns(train.iloc[train_index],features=features).astype(np.float32).values
    X = torch.tensor(Xv)
    yv = train.iloc[train_index][['Survived']].astype(np.float32).values
    y = torch.tensor(yv)
    
    for n in (range(num_epochs)):
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()        
        loss.backward()
        optimizer.step()

    Xv_test = calculate_columns(train.iloc[test_index],features=features).astype(np.float32).values
    X_test = torch.tensor(Xv_test)
    yv_pred = model(X_test)

    acc.append((np.round((yv_pred).detach().cpu().numpy()) ==  train.iloc[test_index][['Survived']].values).mean())    

In [16]:
[print(f"Fold {i} accuracy = {x:.1%}\n") for i,x in enumerate(acc)];

Fold 0 accuracy = 76.0%

Fold 1 accuracy = 80.3%

Fold 2 accuracy = 73.6%

Fold 3 accuracy = 78.7%

Fold 4 accuracy = 61.2%



Hyper parameter tuning

In [22]:
epochs = [100,200,500,1000,2000,5000,10000]
epoch_acc = []
for epoch in epochs:
    nodes = 10
    model = nn.Sequential(nn.Linear(6,nodes),
                          nn.ReLU(),
                          nn.Linear(nodes,1),
                          nn.Sigmoid()
                         )
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    num_epochs = epoch
    
    kf = KFold(n_splits=5,shuffle=True)
    acc = []
    
    features = ['_sex','_age','_fare','_siblings','_par','_class']
    train = preprocess(train)
    
    for i, (train_index, test_index) in enumerate(kf.split(train)):
        Xv = calculate_columns(train.iloc[train_index],features=features).astype(np.float32).values
        X = torch.tensor(Xv)
        yv = train.iloc[train_index][['Survived']].astype(np.float32).values
        y = torch.tensor(yv)
        
        # print(i)
        for n in (range(num_epochs)):
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            optimizer.zero_grad()        
            loss.backward()
            optimizer.step()
    
        Xv_test = calculate_columns(train.iloc[test_index],features=features).astype(np.float32).values
        X_test = torch.tensor(Xv_test)
        yv_pred = model(X_test)
    
        acc.append((np.round((yv_pred).detach().cpu().numpy()) ==  train.iloc[test_index][['Survived']].values).mean())

    epoch_acc.append(np.mean(acc))    

In [24]:
[print(f"Epochs = {x} accuracy = {y:.1%}\n") for x,y in (list(zip(epochs,epoch_acc)))];

Epochs = 100 accuracy = 79.5%

Epochs = 200 accuracy = 80.8%

Epochs = 500 accuracy = 82.0%

Epochs = 1000 accuracy = 81.3%

Epochs = 2000 accuracy = 79.1%

Epochs = 5000 accuracy = 83.0%

Epochs = 10000 accuracy = 80.2%



Train using all data using GPU

In [25]:
model = nn.Sequential(nn.Linear(6,10),
                      nn.ReLU(),
                      nn.Linear(10,1),
                      nn.Sigmoid()
                     ).to(device)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 2000

features = ['_sex','_age','_fare','_siblings','_par','_class']
train = preprocess(train)
Xv = calculate_columns(train,features=features).astype(np.float32).values
X = torch.tensor(Xv,device=device)
yv = train[['Survived']].astype(np.float32).values
y = torch.tensor(yv,device=device)

for n in (range(num_epochs)):
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Predict on test data

In [26]:
test = preprocess(test)

Xvtest = calculate_columns(test,features=features).astype(np.float32).values
Xtest = torch.tensor(Xvtest,device=device)
ytest_pred = model(Xtest)
ytest_pred_arr = np.round((ytest_pred).detach().cpu().numpy())
test['Survived'] = ytest_pred_arr.astype(int)

Export submission

In [28]:
test[['PassengerId','Survived']].to_csv('submission.csv',index=False)